A function for turning structured data into a color-coded latex table.

In [ ]:
import numpy as np


def tabilize(results, precisions, rank_order, suffixes=None, hlines = []):

  def rankify(x, order):
    # Turn a vector of values into a list of ranks, while handling ties.
    assert len(x.shape) == 1
    if order == 0:
      return np.full_like(x, 1e5, dtype=np.int32)
    u = np.sort(np.unique(x))
    if order == 1:
      u = u[::-1]
    r = np.zeros_like(x, dtype=np.int32)
    for ui, uu in enumerate(u):
      mask = x == uu
      r[mask] = ui
    return np.int32(r)

  names = results.keys()
  data = np.array(list(results.values()))
  assert len(names) == len(data)

  tags = [' \cellcolor{tabfirst}',
          '\cellcolor{tabsecond}',
          ' \cellcolor{tabthird}',
          '                     ']

  max_len = max([len(v) for v in list(names)])
  names_padded = [v + ' '*(max_len-len(v)) for v in names]

  data_quant = np.round((data * 10.**(np.array(precisions)[None, :]))) / 10.**(np.array(precisions)[None, :])
  if suffixes is None:
    suffixes = [''] * len(precisions)

  tagranks = []
  for d in range(data_quant.shape[1]):
    tagranks.append(np.clip(rankify(data_quant[:,d], rank_order[d]), 0, len(tags)-1))
  tagranks = np.stack(tagranks, -1)

  for i_row in range(len(names)):
    line = ''
    if i_row in hlines:
      line += '\\hline\n'
    line += names_padded[i_row]
    for d in range(data_quant.shape[1]):
      line += ' & '
      if rank_order[d] != 0 and not np.isnan(data[i_row,d]):
        line += tags[tagranks[i_row, d]]
      if np.isnan(data[i_row,d]):
        line += ' - '
      else:
        assert precisions[d] >= 0
        line += ('{:' + f'0.{precisions[d]}f' + '}').format(data_quant[i_row,d]) + suffixes[d]
    if i_row < len(names)-1:
      line += ' \\\\'
    print(line)

A script for calling tabilize on some placeholder data.

In [ ]:
results = {
    'That one algorithm': [30.52436, 0.151243, 9.61],
    'That other algorithm': [32.1315, 0.074125, 100.1231],
    'Yet another algorithm': [19.26456, 0.43312, 3.10],
    'My beloved algorithm': [38.924123, 0.051241, 60.3145]}
precisions = [2, 3, 0]  # How many digits of precision to use.
rank_order = [1, -1, 0]  # +1 = higher is better, -1 = lower is better, 0 = do not color code.
suffixes = ['', '', ' sec.']  # What string to append after each number.
hlines = [3] # Where to insert horizontal lines.
tabilize(results, precisions, rank_order, suffixes=suffixes, hlines=hlines)

That one algorithm    &  \cellcolor{tabthird}30.52 &  \cellcolor{tabthird}0.151 & 10 sec. \\
That other algorithm  & \cellcolor{tabsecond}32.13 & \cellcolor{tabsecond}0.074 & 100 sec. \\
Yet another algorithm &                      19.26 &                      0.433 & 3 sec. \\
\hline
My beloved algorithm  &  \cellcolor{tabfirst}38.92 &  \cellcolor{tabfirst}0.051 & 60 sec.


And here's some toy latex code that you can dump the above generated latex into:

In [ ]:
\definecolor{tabfirst}{rgb}{1, 0.7, 0.7} % red
\definecolor{tabsecond}{rgb}{1, 0.85, 0.7} % orange
\definecolor{tabthird}{rgb}{1, 1, 0.7} % yellow

\begin{table}
    \centering
    \begin{tabular}{l|ccc}
    & PSNR $\uparrow$ & LPIPS $\downarrow$ & Runtime \\ \hline

PASTE THAT LATEX HERE

    \end{tabular}
    \caption{Wow what great results}
    \label{tab:results}
\end{table}